In [1]:
def gcd(a,b):
    if a==0 or b==0: ## account for a or b == 0
        return max(a,b), (1 if a>0 else 0), (1 if b>0 else 0)   
    u, g, x, y = 1, a, 0, b
    while y!=0:
        q = g//y
        s, t = u-q*x, g%y
        u, g = x, y
        x, y = s, t
    v = (g-a*u)/b

    ## make u as small as possible while u>0
    if u<0:
        u2 = abs(u)
        increase = u2//(b//g) +1
        u+=increase*(b//g)
        v-=increase*(a//g)
        
    print(f'gcd = {g}, u = {u}, v = {int(v)}')
    return u

## Nuestra llave pública

Elegimos $p=6221$, $q=5443$, $e=65537$, para generar nuestra llave pública $(pq, e) = (33860903, 65537)$

In [104]:
p = 6221
q = 5443

n = p*q

e = 65537 

phi_n = (p-1)*(q-1)
print(f'NUESTRA LLAVE PÚBLICA ES {(n, e)}')

NUESTRA LLAVE PÚBLICA ES (33860903, 65537)


## Cálculo de nuestra llave privada

In [82]:
d = gcd(e,phi_n)

gcd = 1, u = 8319113, v = -16107


In [83]:
d*e % phi_n

1

In [84]:
print(f'NUESTRA LLAVE PRIVADA ({d})')

NUESTRA LLAVE PRIVADA (8319113)


## Llave pública de equipo 2

Con esta llave pública, que recuperamos del grupo de Teams, vamos a encriptar un mensaje y mandarlo al equipo 2.

In [95]:
n_2 = 26601037
e_2 = 65537
print(f"LLAVE PÚBLICA DEL EQUIPO 2 {(n_2, e_2)}")

LLAVE PÚBLICA DEL EQUIPO 2 (26601037, 65537)


## Nuestro mensaje a mandar

In [137]:
s = 'cabecita a puertaaaa'

In [138]:
len(s)

20

## Preparación del mensaje

In [112]:
numeros = [ord(x)-96 if x != ' ' else 0 for x in s]

In [113]:
bloques = []
for i in range(4, 24, 4):
    bloques.append((numeros[-4 + i:i]))
bloques

[[3, 1, 2, 5], [3, 9, 20, 1], [0, 1, 0, 16], [21, 5, 18, 20], [1, 1, 1, 1]]

## Cifrado del mensaje

In [114]:
def cifrar_rsa(m, l):
    return m[3] + m[2]*l + m[1]*(l**2) + m[0]*(l**3)

In [117]:
mensaje = []
for i,m in enumerate(bloques): mensaje.append(cifrar_rsa(m, 27))
    
mensaje

[59837, 66151, 745, 417494, 20440]

## Encripción del mensaje

In [120]:
## Encriptar
encriptado = [pow(m,e_2,n_2) for m in mensaje]
encriptado

[24664452, 4295140, 10008554, 12319427, 19778016]

**Este es el mensaje encriptado que mandamos al equipo 2**: `[24664452, 4295140, 10008554, 12319427, 19778016]`

## Desencripción de mensajes

In [20]:
def desencriptar_c(n,d,c):

    m = pow(c, d, n)
    
    return m

In [131]:
def convertir_palabra(num, l):
    letras = []
    while num > 0:
        letra_num = num % l
        if letra_num == 0:
            letra = ' '
        else:
            letra =  chr(letra_num+96)
        
        letras.append(letra)
        num = num // l
    if len(letras) != 4:
        letras.append(' ')
    return ''.join(letras)[::-1]

**Este es el mensaje que recibimos del equipo 3**: `[10061897, 12347467, 11509035, 11114322, 4806613]`

In [139]:
mensaje_mike = [10061897, 12347467, 11509035, 11114322, 4806613]

In [133]:
m = ''.join([convertir_palabra(x, 27) for x in [desencriptar_c(n,d,x) for x in mensaje_mike]])
m 

'leolio boletos akron'

**El mensaje que nos mandaron es 'leolio boletos akron'**

## Volvemos a encriptar este mensaje con la clave pública del equipo que nos lo mandó

In [70]:
# Mensaje cifrado
m_ellos_cifrado = [desencriptar_c(n,d,x) for x in mensaje_mike]
# llave pública de ellos
n_ellos, e_ellos = (6343123, 65537)
# encriptamos el mensaje con su llave pública
m_encriptado = [pow(m, e_ellos, n_ellos) for m in m_ellos_cifrado]

In [71]:
m_encriptado

[3927278, 3300709, 4702408, 6002710, 772705]

El mensaje que nos mandaron lo encriptamos de nuevo, con la llave pública del equipo que nos lo mandó, para obtener el siguiente mensaje encriptado: `[3927278, 3300709, 4702408, 6002710, 772705]`

## Desencriptamos el mensaje que nos devolvieron, que debe ser el que originalmente mandamos

In [134]:
m_devuelto = [19998916, 13114800, 13578549, 27798912, 20926200]
m_original = ''.join([convertir_palabra(x, 27) for x in [desencriptar_c(n,d,x) for x in m_devuelto]])

In [135]:
m_original

'cabecita a puertaaaa'

In [136]:
m_original == s

True

**El mensaje que nos mandaron de regreso sí es el mismo que nosotros mandamos**

## Intentar romper la clave privada de un equipo: equipo 6
Su $N=15142739$

In [76]:
N = 15142739
# fuerza bruta: ver qué número de 2-raiz(N) es factor de N
raiz_N = int(N**(1/2))
p = 0
for i in range(2, raiz_N):
    if N % i == 0:
        p = i
        break

In [77]:
p

2693

Uno de los factores de N es 2693. Con Wolfram verificamos que sí es un número primo.
¿Cuál es el otro factor?

In [78]:
q = N / factor
q

5623.0

$N = pq$, donde $p=2693$, y $q=5623$, con $p,q$ primos.

$\Phi(N) = (p-1)(q-1) = 15134424$

$\Phi(N) = N - (p + q) + 1 = 15142739 - (2693 + 5623) + 1 = 15134424$

In [80]:
((p - 1)*(q - 1)) == (N - (p + q) + 1)

True